<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #5: TF-IDF
`Fecha de entrega: Septiembre 19, 2020. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

### Estudiante: Ilvar Dario Sanabria G. 
### Código      : 616201058

In [7]:
import re
import numpy as np
import pandas as pd

from nltk.corpus import stopwords #Palabras vacias
stopwords_sp = stopwords.words('spanish')

from nltk.stem.snowball import SnowballStemmer #Stemmer
spanishStemmer=SnowballStemmer("spanish")

from sklearn.feature_extraction.text import TfidfVectorizer  #TD-IDF
from sklearn.metrics.pairwise import cosine_distances # Calculo distacia coseno

# Punto 1: Pre-Procesamiento

- `[18 pts]` Leer el archivo `Princesas.csv` usando `pandas` y crear una nueva columna con el texto en minúscula, sin caracteres especiales ni números, sin palabras vacias y hacer stemming de las palabras

In [8]:
corpus = pd.read_csv('c:/tmp/Princesas.csv')
corpus

,Princesa,Personalidad
0,Blancanieves,Blancanieves es una princesa de noble cuna que...
1,Cenicienta,Cenicienta es inicialmente una sirvienta en su...
2,Aurora,"La Princesa Aurora, la Bella Durmiente, es la ..."
3,Bella,Bella es una muchacha que vive en la campiña f...
4,Jasmín,"Cuando se introdujo por primera vez, la Prince..."
5,Pocahontas,"El nombre de Pocahontas significa ""Pequeña Sil..."
6,Mulan,Mulan es atípica a los anteriores papeles feme...
7,Tiana,Es una joven camarera que sueña con ser dueña ...
8,Mérida,Mérida llama la atención por su característico...
9,Moana,"Moana, una joven de 16 años de edad, hija únic..."


In [33]:
# data

,Princesa,Personalidad
0,Blancanieves,Blancanieves es una princesa de noble cuna que...
1,Cenicienta,Cenicienta es inicialmente una sirvienta en su...
2,Aurora,"La Princesa Aurora, la Bella Durmiente, es la ..."
3,Bella,Bella es una muchacha que vive en la campiña f...
4,Jasmín,"Cuando se introdujo por primera vez, la Prince..."
5,Pocahontas,"El nombre de Pocahontas significa ""Pequeña Sil..."
6,Mulan,Mulan es atípica a los anteriores papeles feme...
7,Tiana,Es una joven camarera que sueña con ser dueña ...
8,Mérida,Mérida llama la atención por su característico...
9,Moana,"Moana, una joven de 16 años de edad, hija únic..."


In [9]:
corpus['pre-procesado']=''
i=0
for frase in corpus.Personalidad:
    frase=frase.lower()
    frase=re.sub(r"[\W\d_]+", " ", frase)
    frase = [spanishStemmer.stem(palabra) for palabra in frase.split() if palabra not in stopwords_sp]
    frase= ' '.join(frase)
    corpus['pre-procesado'][i]=frase
    i=i+1
    
corpus

,Princesa,Personalidad,pre-procesado
0,Blancanieves,Blancanieves es una princesa de noble cuna que...,blancaniev princes nobl cun ve forz servidumbr...
1,Cenicienta,Cenicienta es inicialmente una sirvienta en su...,cenicient inicial sirvient cas constant objet ...
2,Aurora,"La Princesa Aurora, la Bella Durmiente, es la ...",princes auror bell durmient hij unic rein flor...
3,Bella,Bella es una muchacha que vive en la campiña f...,bell muchach viv campiñ frances padr inventor ...
4,Jasmín,"Cuando se introdujo por primera vez, la Prince...",introduj primer vez princes jasmin poc dias de...
5,Pocahontas,"El nombre de Pocahontas significa ""Pequeña Sil...",nombr pocahont signif pequeñ silenci bas figur...
6,Mulan,Mulan es atípica a los anteriores papeles feme...,mul atip anterior papel femenin pelicul disney...
7,Tiana,Es una joven camarera que sueña con ser dueña ...,jov camarer sueñ ser dueñ propi restaur algun ...
8,Mérida,Mérida llama la atención por su característico...,mer llam atencion caracterist pel anaranj oscu...
9,Moana,"Moana, una joven de 16 años de edad, hija únic...",moan jov años edad hij unic sucesor import jef...


In [36]:
# data procesada

,Princesa,Personalidad,pre-procesado
0,Blancanieves,Blancanieves es una princesa de noble cuna que...,blancaniev princes nobl cun ve forz servidumbr...
1,Cenicienta,Cenicienta es inicialmente una sirvienta en su...,cenicient inicial sirvient cas constant objet ...
2,Aurora,"La Princesa Aurora, la Bella Durmiente, es la ...",princes auror bell durmient hij unic rein flor...
3,Bella,Bella es una muchacha que vive en la campiña f...,bell muchach viv campiñ frances padr inventor ...
4,Jasmín,"Cuando se introdujo por primera vez, la Prince...",introduj primer vez princes jasmin poc dias de...
5,Pocahontas,"El nombre de Pocahontas significa ""Pequeña Sil...",nombr pocahont signif pequeñ silenci bas figur...
6,Mulan,Mulan es atípica a los anteriores papeles feme...,mul atip anterior papel femenin pelicul disney...
7,Tiana,Es una joven camarera que sueña con ser dueña ...,jov camarer sueñ ser dueñ propi restaur algun ...
8,Mérida,Mérida llama la atención por su característico...,mer llam atencion caracterist pel anaranj oscu...
9,Moana,"Moana, una joven de 16 años de edad, hija únic...",moan jov años edad hij unic sucesor import jef...


# Punto 2: TF-IDF

- `[16 pts]` Crear la matriz TF-IDF

In [13]:
tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(corpus['pre-procesado'].values)

tfidf_matrix = pd.DataFrame(data=tfidf.toarray(), columns=tfidf_vect.get_feature_names())

tfidf_matrix = tfidf_matrix.T.round(3)
tfidf_matrix.columns = corpus.Princesa

tfidf_matrix

Princesa,Blancanieves,Cenicienta,Aurora,Bella,Jasmín,Pocahontas,Mulan,Tiana,Mérida,Moana
abrum,0.000,0.0,0.065,0.000,0.000,0.000,0.0,0.000,0.00,0.000
abuel,0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.102
acuerd,0.000,0.0,0.056,0.000,0.000,0.000,0.0,0.000,0.08,0.000
ademas,0.000,0.0,0.000,0.000,0.000,0.099,0.0,0.000,0.00,0.000
afortun,0.087,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.000
...,...,...,...,...,...,...,...,...,...,...
viv,0.000,0.0,0.000,0.074,0.000,0.000,0.0,0.000,0.00,0.087
volunt,0.000,0.0,0.000,0.000,0.161,0.000,0.0,0.000,0.00,0.000
volv,0.000,0.0,0.065,0.000,0.000,0.000,0.0,0.000,0.00,0.000
vudu,0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.122,0.00,0.000


In [44]:
# tfidf_matrix

,Blancanieves,Cenicienta,Aurora,Bella,Jasmín,Pocahontas,Mulan,Tiana,Mérida,Moana
abrum,0.000,0.0,0.065,0.000,0.000,0.000,0.0,0.000,0.00,0.000
abuel,0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.102
acuerd,0.000,0.0,0.056,0.000,0.000,0.000,0.0,0.000,0.08,0.000
ademas,0.000,0.0,0.000,0.000,0.000,0.099,0.0,0.000,0.00,0.000
afortun,0.087,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.000
...,...,...,...,...,...,...,...,...,...,...
viv,0.000,0.0,0.000,0.074,0.000,0.000,0.0,0.000,0.00,0.087
volunt,0.000,0.0,0.000,0.000,0.161,0.000,0.0,0.000,0.00,0.000
volv,0.000,0.0,0.065,0.000,0.000,0.000,0.0,0.000,0.00,0.000
vudu,0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.122,0.00,0.000


# Punto 3: Distancia del coseno
- `[12 pts]` Calcular la distancia del coseno entre cada una de las princesas
- `[2 pts]` ¿Cuáles son las princesas más parecidas?
- `[2 pts]` ¿Cuáles son las princesas más diferentes?

In [ ]:
#### Distacia del coseno 

In [15]:
dist_cos = cosine_distances(tfidf_matrix.T.values)
dist_cos = pd.DataFrame(dist_cos, columns = tfidf_matrix.columns, index = tfidf_matrix.columns)
dist_cos

Princesa,Blancanieves,Cenicienta,Aurora,Bella,Jasmín,Pocahontas,Mulan,Tiana,Mérida,Moana
Princesa,,,,,,,,,,
Blancanieves,0.000000,0.846420,0.842223,0.918112,0.935103,0.922165,0.884147,0.960082,0.985136,0.951298
Cenicienta,0.846420,0.000000,0.858412,0.947243,0.959209,0.960479,0.932663,0.918639,0.970839,0.974925
Aurora,0.842223,0.858412,0.000000,0.832503,0.937195,0.868994,0.919500,0.939231,0.940723,0.965930
Bella,0.918112,0.947243,0.832503,0.000000,0.888219,0.890513,0.882169,0.973338,0.943607,0.923546
Jasmín,0.935103,0.959209,0.937195,0.888219,0.000000,0.942753,0.935198,0.937868,0.972505,0.968929
Pocahontas,0.922165,0.960479,0.868994,0.890513,0.942753,0.000000,0.888844,0.968918,0.979646,0.936091
Mulan,0.884147,0.932663,0.919500,0.882169,0.935198,0.888844,0.000000,0.991170,0.987267,0.945844
Tiana,0.960082,0.918639,0.939231,0.973338,0.937868,0.968918,0.991170,0.000000,0.979275,0.971443
Mérida,0.985136,0.970839,0.940723,0.943607,0.972505,0.979646,0.987267,0.979275,0.000000,0.971818


In [47]:
# dist_cos

,Blancanieves,Cenicienta,Aurora,Bella,Jasmín,Pocahontas,Mulan,Tiana,Mérida,Moana
Blancanieves,0.000000,0.846420,0.842223,0.918112,0.935103,0.922165,0.884147,0.960082,0.985136,0.951298
Cenicienta,0.846420,0.000000,0.858412,0.947243,0.959209,0.960479,0.932663,0.918639,0.970839,0.974925
Aurora,0.842223,0.858412,0.000000,0.832503,0.937195,0.868994,0.919500,0.939231,0.940723,0.965930
Bella,0.918112,0.947243,0.832503,0.000000,0.888219,0.890513,0.882169,0.973338,0.943607,0.923546
Jasmín,0.935103,0.959209,0.937195,0.888219,0.000000,0.942753,0.935198,0.937868,0.972505,0.968929
Pocahontas,0.922165,0.960479,0.868994,0.890513,0.942753,-0.000000,0.888844,0.968918,0.979646,0.936091
Mulan,0.884147,0.932663,0.919500,0.882169,0.935198,0.888844,0.000000,0.991170,0.987267,0.945844
Tiana,0.960082,0.918639,0.939231,0.973338,0.937868,0.968918,0.991170,0.000000,0.979275,0.971443
Mérida,0.985136,0.970839,0.940723,0.943607,0.972505,0.979646,0.987267,0.979275,0.000000,0.971818
Moana,0.951298,0.974925,0.965930,0.923546,0.968929,0.936091,0.945844,0.971443,0.971818,0.000000


In [16]:
print (f"Las princesas mas parecidas son Tiana y Mulan con una distancia de {dist_cos.values.max().round(3)}")

Las princesas mas parecidas son Tiana y Mulan con una distancia de 0.991


In [21]:
dist_cos

Princesa,Blancanieves,Cenicienta,Aurora,Bella,Jasmín,Pocahontas,Mulan,Tiana,Mérida,Moana
Princesa,,,,,,,,,,
Blancanieves,0.000000,0.846420,0.842223,0.918112,0.935103,0.922165,0.884147,0.960082,0.985136,0.951298
Cenicienta,0.846420,0.000000,0.858412,0.947243,0.959209,0.960479,0.932663,0.918639,0.970839,0.974925
Aurora,0.842223,0.858412,0.000000,0.832503,0.937195,0.868994,0.919500,0.939231,0.940723,0.965930
Bella,0.918112,0.947243,0.832503,0.000000,0.888219,0.890513,0.882169,0.973338,0.943607,0.923546
Jasmín,0.935103,0.959209,0.937195,0.888219,0.000000,0.942753,0.935198,0.937868,0.972505,0.968929
Pocahontas,0.922165,0.960479,0.868994,0.890513,0.942753,0.000000,0.888844,0.968918,0.979646,0.936091
Mulan,0.884147,0.932663,0.919500,0.882169,0.935198,0.888844,0.000000,0.991170,0.987267,0.945844
Tiana,0.960082,0.918639,0.939231,0.973338,0.937868,0.968918,0.991170,0.000000,0.979275,0.971443
Mérida,0.985136,0.970839,0.940723,0.943607,0.972505,0.979646,0.987267,0.979275,0.000000,0.971818


In [36]:
min =1
for i_fila,fila in dist_cos.iterrows():
    for valor in fila:
        if valor >0 and valor < min : min = valor
# if i.values<min and i.values>0: min =i.values
        
np.where(dist_cos == min)

(array([2, 3], dtype=int64), array([3, 2], dtype=int64))

In [38]:
print (f"Las princesas menos parecidas son Aurora y Bancanieves: {round(min,3)}")

Las princesas menos parecidas son Aurora y Bancanieves: 0.833
